<a href="https://colab.research.google.com/github/champsleague/DeepLearning/blob/main/DL_Lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
class myRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(myRNN, self).__init__()

    self.input_size = input_size # length of one-hot encoding vector
    self.hidden_size = hidden_size # hidden state size which is determined by a designer
    self.output_size = output_size # length of on-hot encoding vector
   
    # U, W, V are implemented by fully-connected layers
    self.U = nn.Linear(input_size, hidden_size)
    self.W = nn.Linear(hidden_size, hidden_size)
    self.V = nn.Linear(hidden_size, output_size)
    
    self.activation_func = nn.Tanh() # activation cuntion = tanh 
    self.softmax = nn.LogSoftmax(dim=0) # Softmax function contains Cross-Entropy Loss

  def forward(self, input, hidden):
    hidden = self.activation_func(self.U(input) + self.W(hidden)) # ht = tanh(U*xt + W*ht+1)
    output = self.V(hidden) # softmax(V*ht)
    return output, hidden
    
  def init_hidden(self):
    return torch.zeros(1, self.hidden_size) # initialization of the first hidden values 

In [4]:
# define character list as the string type 
chars = "abcdefghijklmnopqrstuvwxyz .,:;?01" # index 0123456 .......

# convert character list to list type
char_list = [i for i in chars] 

# get the length of character list
n_letters = len(char_list)

In [5]:
# For Input 
def word_to_onehot(string):
  # we will use start and end code by using 0, 1
  start = np.zeros(shape=len(char_list), dtype=int)
  end = np.zeros(shape=len(char_list), dtype=int)
  start[-2] = 1
  end[-1] = 1
  
  for i in string:
  # one-hot encoding of training string will be located between start and end vector
    idx = char_list.index(i)
    zero = np.zeros(shape=n_letters, dtype=int)
    zero[idx] = 1
    start = np.vstack([start, zero])
  output = np.vstack([start, end])
  return output

In [6]:
# For Output / Test 
def onehot_to_word(onehot_1): 
  onehot = torch.Tensor.numpy(onehot_1)
  return char_list[onehot.argmax()]

In [22]:
def main():
  n_hidden = 128 # set hidden size 128 
  lr = 0.0001 # set learning rate to 0.001 
  epochs = 10000 # set epochs to 1000 (5000)
  
  # set training string
  # < Question 1 > 
  # string = "i want to go on a trip these days. how about you?"
  
  # < Question 2 > 
  string = "i want to go on a trip these days. how about you? i would like to visit france, italy, germany again with my friends."
  
  # initiate RNN
  rnn = myRNN(n_letters, n_hidden, n_letters)
   
  loss_func = nn.CrossEntropyLoss()  # use Cross-Entropy Loss as loss function 
  optimizer = torch.optim.Adam(rnn.parameters(), lr=lr) # Adam optimizer 
  
  ## Training 
  one_hot = torch.from_numpy(word_to_onehot(string)).type_as(torch.FloatTensor())
  
  for i in range(epochs):
    optimizer.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    input_ = one_hot[0:1, :]
    
    for j in range(one_hot.size()[0]-1):
      target = one_hot[j+1]
      target_single = torch.from_numpy(np.asarray(target.numpy().argmax())).type_as(torch.LongTensor()).view(-1)
      
      output, hidden = rnn.forward(input_, hidden)
      loss = loss_func(output, target_single)
      total_loss += loss
      input_ = output
    
    total_loss.backward()
    optimizer.step()
    
    if i%100 == 0:
      print('epoch%d'%i)
      print(total_loss)
       
  ## Test
  start = torch.zeros(1, len(char_list))
  start[:,-2] = 1
  
  with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    
    for i in range(len(string)):
      output, hidden = rnn.forward(input_, hidden)
      output_string += onehot_to_word(F.softmax(output.data))
      input_ = output
      
    print(output_string)


# Run the code

In [23]:
main()

epoch0
tensor(415.5131, grad_fn=<AddBackward0>)
epoch100
tensor(352.6122, grad_fn=<AddBackward0>)
epoch200
tensor(346.2035, grad_fn=<AddBackward0>)
epoch300
tensor(343.4576, grad_fn=<AddBackward0>)
epoch400
tensor(339.4114, grad_fn=<AddBackward0>)
epoch500
tensor(334.4246, grad_fn=<AddBackward0>)
epoch600
tensor(327.3868, grad_fn=<AddBackward0>)
epoch700
tensor(322.9275, grad_fn=<AddBackward0>)
epoch800
tensor(317.8551, grad_fn=<AddBackward0>)
epoch900
tensor(313.3098, grad_fn=<AddBackward0>)
epoch1000
tensor(308.5993, grad_fn=<AddBackward0>)
epoch1100
tensor(328.3262, grad_fn=<AddBackward0>)
epoch1200
tensor(311.1369, grad_fn=<AddBackward0>)
epoch1300
tensor(307.5543, grad_fn=<AddBackward0>)
epoch1400
tensor(304.2137, grad_fn=<AddBackward0>)
epoch1500
tensor(301.9839, grad_fn=<AddBackward0>)
epoch1600
tensor(299.7937, grad_fn=<AddBackward0>)
epoch1700
tensor(297.6776, grad_fn=<AddBackward0>)
epoch1800
tensor(295.4466, grad_fn=<AddBackward0>)
epoch1900
tensor(293.1254, grad_fn=<AddBack

<ipython-input-22-ba750b53b944>:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output_string += onehot_to_word(F.softmax(output.data))


# Result Analysis

GT = i want to go on a trip theses days. how about you?

How can we improve the performace ? 


# Try many other options

In [9]:
# increase number of hidden layer 
n_hidden = 256 

# increase epochs
epochs = 5000 

# etc... 